![](imgs/intro_01.png)

# Detección de intrusos

## Objetivo

Este notebook tiene como objetivo mostrar el uso del servicio cognitivo **Face API de Microsoft Azure** para generar una "base de datos" de rostros y después comparar nuevos rostros. 

## Temas

1. Activación del servicio en el portal Azure
2. Creación de base de datos y usuario
3. Carga de información a la base de datos
4. Comparación de nuevos rostros

## Requisitos

- Cuenta de Azure
- Python 3.x
- Librerias:
    - requests
    - os
    - time

## 1. Activación del servicio en el portal Azure

1. Ingresar a la dirección [portal.azure.com](https://portal.azure.com)

2. Buscar **Cognitive services**

![](imgs/paso_01.png)

3. Seleccionar **Add**

![](imgs/paso_02.png)

4. Buscar el servicio de su elección. En este caso es **Face**

![](imgs/paso_03.png)

5. Seleccionar **Create**

![](imgs/paso_04.png)

6. Llenar el formulario con la información solicitada y seleccionar **Create**.

Un ejemplo sería:
    - Name: deteccion-intruso
    - Subscription: Azure for Students
    - Location: (US) East US 2
    - Pricing tier: S0 (10 Calls per Second)
    - Resource group: clase_azure_unam
    
![](imgs/paso_05.png)

7. Esperar alrededor de un mintuo hasta que indique que el "despliegue" del servicio está completo. Seleccionar **Go to resource**.

![](imgs/paso_06.png)

8. En esta sección se pueden observar tanto el **key** como el **endpoint**, los cuales son necesarios para realizar los llamados al API.

![](imgs/paso_07.png)

9. Copiar y guardar la información de **key** y **endpoint** para usarlo en python.

En este ejemplo se creó un archivo **conf_param.py** que contiene la siguiente información

```python
KEY = 'key'
BASE_URL = 'endpoint'
```

10. Guardar el archivo en la misma ubicación que este notebook.

## 2. Creación de la base de datos y usuarios

Una vez hemos desplegado el servicio cognitivo **Face API** y contamos con la **key** y **endpoint** del mismo, procedemos a abrir una consola de jupyter notebook para comenzar a configurar la base de datos que almacenará las imágenes que utilizaremos para determinar si una persona tiene permitido el acceso o no.

Para este tutorial se utiliará información proveniente de la documentación de Microsoft para [Face API](https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/563879b61984550f30395236)

1. Cargar las librerias e información a utilizar.

In [7]:
import requests, os, time
KEY = ''
END_POINT = ''
BASE_URL = END_POINT+'/face/v1.0'

2. Definimos el nombre de la base de datos a utilizar.

In [8]:
nombre_base = 'demo_intrusos'

3. Procedemos a crear la base de datos

Para esto, se utilizará la sección de **LargePersonGroup** mostrada en la [documentación](https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/599acdee6ac60f11b48b5a9d)

A continuación se muestra una parte de dicha documentación donde nos explica los parámetros a utilizar.

### LargePersonGroup - Create

Create a new large person group with user-specified largePersonGroupId, name, an optional userData and recognitionModel.
A large person group is a container holding the uploaded person data, including the face recognition features. It can hold up to 1,000,000 entities.
After creation, use LargePersonGroup Person - Create to add person into the group, and call LargePersonGroup - Train to get this group ready for Face - Identify.
No image will be stored. Only the person's extracted face feature(s) and userData will be stored on server until LargePersonGroup Person - Delete or LargePersonGroup - Delete is called.

'recognitionModel' should be specified to associate with this large person group. The default value for 'recognitionModel' is 'recognition_01', if the latest model needed, please explicitly specify the model you need in this parameter. New faces that are added to an existing large person group will use the recognition model that's already associated with the collection. Existing face feature(s) in a large person group can't be updated to features extracted by another version of recognition model.

* 'recognition_01': The default recognition model for LargePersonGroup - Create. All those large person groups created before 2019 March are bonded with this recognition model.
* 'recognition_02': Recognition model released in 2019 March. 'recognition_02' is recommended since its overall accuracy is improved compared with 'recognition_01'.

Large person group quota:
* Free-tier subscription quota: 1,000 large person groups.
* S0-tier subscription quota: 1,000,000 large person groups.

#### Http Method

PUT

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
|largePersonGroupId | string | User-provided largePersonGroupId as a string. The valid characters include numbers, English letters in lower case, '-' and '_'. The maximum length of the largePersonGroupId is 64. |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Content-Type | string | Media type of the body sent to the API. |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

JSON fields in request body:

| Fields | Type | Description |
|:--- |:---:|:--- |
| name | String | Name of the created large person group, maximum length is 128. |
| userData (optional) | String | Optional user defined data for the large person group. Length should not exceed 16KB. |
| recognitionModel (optional) | String | The 'recognitionModel' associated with this large person group. Supported 'recognitionModel' values include "recognition_01" and "recognition_02". The default value is "recognition_01". "recognition_02" is recommended since its overall accuracy is improved compared with "recognition_01". | 

In [1]:
response = requests.request(
    #method
    'PUT',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base,
    #json
    json = {'name': nombre_base,
            'recognitionModel': 'recognition_02'},
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

NameError: name 'requests' is not defined

4. El código anterior debe dar como resultado un **200**. En caso contrario validar los pasos previos.

5. Podemos confirmar que la base se creo de forma correcta solicitando la lista de todas las bases de datos.

### LargePersonGroup - List

List all existing large person groups’s largePersonGroupId, name, userData and recognitionModel.
* Large person groups are stored in alphabetical order of largePersonGroupId.
* "start" parameter (string, optional) is a user-provided largePersonGroupId value that returned entries have larger ids by string comparison. "start" set to empty to indicate return from the first item.
* "top" parameter (int, optional) specifies the number of entries to return. A maximal of 1000 entries can be returned in one call. To fetch more, you can specify "start" with the last retuned entry’s Id of the current call.

For example, total 5 large person groups: "group1", ..., "group5".
"start=&top=" will return all 5 groups.
"start=&top=2" will return "group1", "group2".
"start=group2&top=3" will return "group3", "group4", "group5".

#### Http Method

GET

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups[?start][&top][&returnRecognitionModel]

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
| start (optional) | string | List large person groups from the least largePersonGroupId greater than the "start". It contains no more than 64 characters. Default is empty. |
| top (optional) | integer | The number of large person groups to list, ranging in [1, 1000]. Default is 1000. |
| returnRecognitionModel (optional) | boolean | Return 'recognitionModel' or not. The default value is false. |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body


In [37]:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/',
    #params
    params = {'returnRecognitionModel': True},
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

6. El código anterior debe dar como resultado un **200**. De tal forma que podemos ver la información que regresa como se muestra a continuación:

In [38]:
response.json()

[{'recognitionModel': 'recognition_02',
  'largePersonGroupId': 'demo_intrusos',
  'name': 'demo_intrusos',
  'userData': None}]

7. También podemos verificar el status de la base para saber si ya ha sido entrenada o no.

### LargePersonGroup - Get Training Status

To check large person group training status completed or still ongoing. LargePersonGroup Training is an asynchronous operation triggered by LargePersonGroup - Train API.

Training time depends on the number of person entries, and their faces in a large person group. It could be in seconds, or up to half an hour for 1,000,000 persons.

#### Http Method

GET

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}/training

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
|largePersonGroupId | string | LargePersonGroupId of target person group. |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

In [39]:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/training',
    #headers
    headers = {'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

404

8. El código anterior debe dar como resultado un **404** ya que aún no ha sido entrenado (una vez lo entrenemos debe dar un 200). De tal forma que podemos ver la información que regresa como se muestra a continuación:

In [40]:
response.json()

{'error': {'code': 'LargePersonGroupNotTrained',
  'message': 'Large person group not trained.'}}

9. Para comenzar a cargar información a la base de datos es necesario crear un "contenedor" por cada usuario/persona para que pueda almacenar sus rostros.

### LargePersonGroup Person - Create

Create a new person in a specified large person group. To add face to this person, please call LargePersonGroup PersonFace - Add.

- Free-tier subscription quota:
    - 1,000 persons in all large person groups.
- S0-tier subscription quota:
    - 1,000,000 persons per large person group.
    - 1,000,000 large person groups.
    - 1,000,000,000 persons in all large person groups.

### Http Method
POST

### Request URL
https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}/persons

### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
| largePersonGroupId | string | largePersonGroupId of the target large person group. |

### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Content-Type | string | Media type of the body sent to the API. |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

### Request body

JSON fields in request body:

| Field | Type | Description |
|:--- |:---:|:--- |
| name | String | Name of the created person, maximum length is 128. |
| userData (optional) | String | Optional user defined data for the person. Length should not exceed 16KB. |

In [41]:
usuario = 'usuario_demo'

response = requests.request(
    #method
    'POST',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/persons/',
    #json
    json = {'name': usuario},
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

10. En este caso es necesario guardar la respuesta del "request" ya que nos regresa información sobre el ID asociado al usuario. Esta información es necesaria para cargar los rostros relacionados con el usuario.

In [42]:
print(response.json())

personId = response.json()['personId']

{'personId': '8684eddf-c898-47ee-ac6e-98c56eb3c1d9'}


11. Otra forma de obtener esta información es mediante la consulta de todos los usuarios pertenecientes a la base de datos

### LargePersonGroup Person - List

List all persons’ information in the specified large person group, including personId, name, userData and persistedFaceIds of registered person faces.
- Persons are stored in alphabetical order of personId created in LargePersonGroup Person - Create.
- "start" parameter (string, optional) is a personId value that returned entries have larger ids by string comparison. "start" set to empty to indicate return from the first item.
- "top" parameter (int, optional) specifies the number of entries to return. A maximal of 1000 entries can be returned in one call. To fetch more, you can specify "start" with the last returned entry’s personId of the current call.

For example, total 5 persons with their personId: "personId1", ..., "personId5".
"start=&top=" will return all 5 persons.
"start=&top=2" will return "personId1", "personId2".
"start=personId2&top=3" will return "personId3", "personId4", "personId5".

#### Http Method
GET

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}/persons[?start][&top]

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
|largePersonGroupId | string | largePersonGroupId of the target large person group. |
| start (optional) | string | List persons from the least personId greater than the "start". It contains no more than 64 characters. Default is empty. |
| top (optional) | integer | The number of persons to list, ranging in [1, 1000]. Default is 1000. |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

In [43]:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/persons',
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

Y obtenemos la información de los usuarios

In [44]:
response.json()

[{'personId': '8684eddf-c898-47ee-ac6e-98c56eb3c1d9',
  'persistedFaceIds': [],
  'name': 'usuario_demo',
  'userData': None}]

11. Una vez creado el "contenedor" para el usuario, es necesario cargar la base de datos con los vectores de características de los rostros. Es decir, necesitamos cargar las imágenes al API para que las procese.

### LargePersonGroup Person - Add Face

Add a face to a person into a large person group for face identification or verification. To deal with an image containing multiple faces, input face can be specified as an image with a targetFace rectangle. It returns a persistedFaceId representing the added face. No image will be stored. Only the extracted face feature(s) will be stored on server until LargePersonGroup PersonFace - Delete, LargePersonGroup Person - Delete or LargePersonGroup - Delete is called.
Note persistedFaceId is different from faceId generated by Face - Detect.
- Higher face image quality means better recognition precision. Please consider high-quality faces: frontal, clear, and face size is 200x200 pixels (100 pixels between eyes) or bigger.
- Each person entry can hold up to 248 faces.
- JPEG, PNG, GIF (the first frame), and BMP format are supported. The allowed image file size is from 1KB to 6MB.
- "targetFace" rectangle should contain one face. Zero or multiple faces will be regarded as an error. If the provided "targetFace" rectangle is not returned from Face - Detect, there’s no guarantee to detect and add the face successfully.
- Out of detectable face size (36x36 - 4096x4096 pixels), large head-pose, or large occlusions will cause failures.
- Adding/deleting faces to/from a same person will be processed sequentially. Adding/deleting faces to/from different persons are processed in parallel.
- The minimum detectable face size is 36x36 pixels in an image no larger than 1920x1080 pixels. Images with dimensions higher than 1920x1080 pixels will need a proportionally larger minimum face size.
- Different 'detectionModel' values can be provided. To use and compare different detection models, please refer to How to specify a detection model
    - 'detection_01': The default detection model for LargePersonGroup Person - Add Face. Recommend for near frontal face detection. For scenarios with exceptionally large angle (head-pose) faces, occluded faces or wrong image orientation, the faces in such cases may not be detected.
    - 'detection_02': Detection model released in 2019 May with improved accuracy especially on small, side and blurry faces.

#### Http Method

POST

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}/persons/{personId}/persistedfaces[?userData][&targetFace][&detectionModel]

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
| largePersonGroupId | string | largePersonGroupId of the target large person group. |
| personId | string | personId of the target person. |
| userData (optional) | string | User-specified data about the target face to add for any purpose. The maximum length is 1KB. |
| targetFace (optional) | string | A face rectangle to specify the target face to be added to a person, in the format of "targetFace=left,top,width,height". E.g. "targetFace=10,10,100,100". If there is more than one face in the image, targetFace is required to specify which face to add. No targetFace means there is only one face detected in the entire image. |
| detectionModel (optional) | string | The 'detectionModel' associated with the detected faceIds. Supported 'detectionModel' values include "detection_01" or "detection_02". The default value is "detection_01". |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Content-Type | string | Media type of the body sent to the API. |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

JSON fields in request body:

| Field | Type | Description |
|:--- |:---:|:--- |
| url | String | Face image URL. Valid image size is from 1KB to 6MB. Only one face is allowed per image. |

In [45]:
response = requests.request(
    #method
    'POST',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/persons/' + personId + '/persistedFaces',
    # params
    params = {'detectionModel': 'detection_02'},
    #data
    data = open('foto_07.jpg', 'rb').read(),
    #headers
    headers = {'Content-Type': 'application/octet-stream',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

12. Al regresarnos un código **200** sabemos que la identificación del rostro y extracción del vector de características se realizo correctamente.

---
# Ejemplo/Ejercicio

Se creará una base de datos de nombre 'games_of_thrones' y se cargarán las imágenes contenidas en la carpeta de entrenamiento.

In [8]:
# Se crea la base de datos
nombre_base = 'games_of_thrones'
requests.request(
    #method
    'PUT',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base,
    #json
    json = {'name': nombre_base,
            'recognitionModel': 'recognition_02'},
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

# Se genera la lista de usuarios
path_usuario = 'base_rostros/entrenamiento/'
usuarios = os.listdir(path_usuario)

for usuario in usuarios:
    response = requests.request(
        #method
        'POST',
        #url
        BASE_URL + '/largepersongroups/' + nombre_base + '/persons/',
        #json
        json = {'name': usuario},
        #headers
        headers = {'Content-Type': 'application/json',
                   'Ocp-Apim-Subscription-Key': KEY}
    )
    
    personId = response.json()['personId']
    
    # Se genera la lista de fotos del usuario
    path_img = path_usuario + usuario + '/'
    fotos = list(filter(lambda file: file.endswith('.jpg'), os.listdir(path_img)))
    
    for foto in fotos:
        response = requests.request(
        #method
        'POST',
        #url
        BASE_URL + '/largepersongroups/' + nombre_base + '/persons/' + personId + '/persistedFaces',
        # params
        params = {'detectionModel': 'detection_02'},
        #data
        data = open(path_img + foto, 'rb').read(),
        #headers
        headers = {'Content-Type': 'application/octet-stream',
                   'Ocp-Apim-Subscription-Key': KEY}
        )
       
    print('Finaliza la carga de las fotos de ' + usuario)
    time.sleep(30)

# Revisamos los usuarios cargados:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/persons',
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.json()

Finaliza la carga de las fotos de emilia_clarke
Finaliza la carga de las fotos de kit_harington
Finaliza la carga de las fotos de peter_dinklage
Finaliza la carga de las fotos de sophie_turner
Finaliza la carga de las fotos de vladimir_furdik


[{'personId': '0ec895a7-3a1e-47fb-9b3a-576d5c58f91e',
  'persistedFaceIds': ['08c25e6a-5373-4a92-a1cd-f0c3c8140a71',
   '0d267a17-ca2b-455d-899c-4afc6b2286d9',
   '1d891b2d-32cb-4e7c-9cba-2d6ecabef826',
   '21f9feda-2c64-44d1-928d-e3ac64248f5b',
   '2d341c97-d9b0-4393-8b39-87242395c2e8'],
  'name': 'kit_harington',
  'userData': None},
 {'personId': '19ba06ce-cba2-4ded-9bc5-b42d428f2051',
  'persistedFaceIds': ['1519633d-71d3-4757-be3d-c525b3553098',
   '1d6c8c5e-b4ac-4644-a360-fcd9154571e1',
   '1e2e0000-face-4bc9-8870-b676bd8e7c8c',
   '76889b5c-49c5-4d21-b3b6-238e6f90e2f7'],
  'name': 'peter_dinklage',
  'userData': None},
 {'personId': '326788e9-f210-4c32-835a-246d018efe83',
  'persistedFaceIds': ['2d252fe9-efed-42bb-b590-5b7a58e6711a',
   '5655e1d8-387a-4b18-afd6-2eb77978ab2d',
   'adca698b-3866-4291-87ed-915605d16faf',
   'c9c910cf-ec0a-4ea6-a164-a63cf9be1398',
   'f2a03313-fe9b-4771-8cff-895e8d48d0b8'],
  'name': 'emilia_clarke',
  'userData': None},
 {'personId': '66a70c0e-c640

Con esto termina el ejemplo/ejercicio, pero se usará esta base de rostros para los que resta del tutorial.

---

13. Ahora es necesario entrenar la base de datos para que pueda identificar a las personas en ellas.

### LargePersonGroup - Train

Submit a large person group training task. Training is a crucial step that only a trained large person group can be used by Face - Identify.

The training task is an asynchronous task. Training time depends on the number of person entries, and their faces in a large person group. It could be in several seconds, or up to half a hour for 1,000,000 persons. To check training completion, please use LargePersonGroup - Get Training Status.

#### Http Method
POST

#### Request URL

https://{endpoint}/face/v1.0/largepersongroups/{largePersonGroupId}/train

#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
| largePersonGroupId | string | Target large person group to be trained. |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body


In [9]:
response = requests.request(
    #method
    'POST',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/train',
    #headers
    headers = {'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

202

14. En este caso, de acuerdo con la documentación, un código **202** indica una respuesta exitosa y no regresa información en el JSON.

15. Sin embargo, es posible ver el estado de la base de datos usando el llamado para **get trainning status**, el cual vimos previamente.

In [10]:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/training',
    #headers
    headers = {'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

16. Ahora obtenemos una respuesta **200** (diferente a la 404 inicial). Esto implica que el entrenamiento fue correcto.

17. Podemos obtener más información revisando la respuesta del JSON.

In [11]:
response.json()

{'status': 'succeeded',
 'createdDateTime': '2019-10-18T16:44:10.152627Z',
 'lastActionDateTime': '2019-10-18T16:44:10.3400568Z',
 'message': None,
 'lastSuccessfulTrainingDateTime': '2019-10-18T16:44:10.3400568Z'}

18. En el JSON previo observamos el status **succeeded** y nos da fecha y tiempo en que se creo la base, se realizó algún cambio y la última vez que fue entrenado exitosamente.

19. Ahora, con la base entrenada, podemos comenzar a comparar rostros de otras fotos.

### Face - Detect

Detect human faces in an image, return face rectangles, and optionally with faceIds, landmarks, and attributes.

- No image will be stored. Only the extracted face feature(s) will be stored on server. The faceId is an identifier of the face feature and will be used in Face - Identify, Face - Verify, and Face - Find Similar. The stored face features will expire and be deleted 24 hours after the original detection call.
- Optional parameters include faceId, landmarks, and attributes. Attributes include age, gender, headPose, smile, facialHair, glasses, emotion, hair, makeup, occlusion, accessories, blur, exposure and noise. Some of the results returned for specific attributes may not be highly accurate.
- JPEG, PNG, GIF (the first frame), and BMP format are supported. The allowed image file size is from 1KB to 6MB.
- The minimum detectable face size is 36x36 pixels in an image no larger than 1920x1080 pixels. Images with dimensions higher than 1920x1080 pixels will need a proportionally larger minimum face size.
- Up to 100 faces can be returned for an image. Faces are ranked by face rectangle size from large to small.
- For optimal results when querying Face - Identify, Face - Verify, and Face - Find Similar ('returnFaceId' is true), please use faces that are: frontal, clear, and with a minimum size of 200x200 pixels (100 pixels between eyes).
- Different 'detectionModel' values can be provided. To use and compare different detection models, please refer to How to specify a detection model
    - 'detection_01': The default detection model for Face - Detect. Recommend for near frontal face detection. For scenarios with exceptionally large angle (head-pose) faces, occluded faces or wrong image orientation, the faces in such cases may not be detected.
    - 'detection_02': Detection model released in 2019 May with improved accuracy especially on small, side and blurry faces. Face attributes and landmarks are disabled if you choose this detection model.
- Different 'recognitionModel' values are provided. If follow-up operations like Verify, Identify, Find Similar are needed, please specify the recognition model with 'recognitionModel' parameter. The default value for 'recognitionModel' is 'recognition_01', if latest model needed, please explicitly specify the model you need in this parameter. Once specified, the detected faceIds will be associated with the specified recognition model. More details, please refer to How to specify a recognition model.
    - 'recognition_01': The default recognition model for Face - Detect. All those faceIds created before 2019 March are bonded with this recognition model.
    - 'recognition_02': Recognition model released in 2019 March. 'recognition_02' is recommended since its overall accuracy is improved compared with 'recognition_01'.

#### Http Method
POST

#### Request URL

https://{endpoint}/face/v1.0/detect[?returnFaceId][&returnFaceLandmarks][&returnFaceAttributes][&recognitionModel][&returnRecognitionModel][&detectionModel]


#### Request parameters

| Field | Type | Description |
|:--- |:---:|:--- |
| returnFaceId (optional) | boolean | Return faceIds of the detected faces or not. The default value is true. |
| returnFaceLandmarks (optional) | boolean | Return face landmarks of the detected faces or not. The default value is false. |
| returnFaceAttributes (optional) | string | Analyze and return the one or more specified face attributes in the comma-separated string like "returnFaceAttributes=age,gender". Supported face attributes include age, gender, headPose, smile, facialHair, glasses, emotion, hair, makeup, occlusion, accessories, blur, exposure and noise. Face attribute analysis has additional computational and time cost. |
| recognitionModel (optional) | string | The 'recognitionModel' associated with the detected faceIds. Supported 'recognitionModel' values include "recognition_01" or "recognition_02". The default value is "recognition_01". "recognition_02" is recommended since its overall accuracy is improved compared with "recognition_01". |
| returnRecognitionModel (optional) | boolean | Return 'recognitionModel' or not. The default value is false. |
| detectionModel (optional) | string | The 'detectionModel' associated with the detected faceIds. Supported 'detectionModel' values include "detection_01" or "detection_02". The default value is "detection_01". |

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Content-Type | string | Media type of the body sent to the API. |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

To detect in a URL (or binary data) specified image.

JSON fields in the request body:

| Field | Type | Description |
|:--- |:---:|:--- |
| url | String | URL of input image. |

In [111]:
response = requests.request(
    #method
    'POST',
    #url
    BASE_URL + '/detect',
    #params
    params = {'returnFaceId': True,
              'returnFaceLandmarks': False,
              'recognitionModel': 'recognition_02'},
    #data
    data = open('foto_06.jpg', 'rb').read(),
    #headers
    headers = {'Content-Type': 'application/octet-stream',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

20. Obteniendo una respuesta correcta, procedemos a extraer el 'faceId' generado para poder hacer las comparaciones.

In [112]:
print(response.json())

faceId = response.json()[0]['faceId']

[{'faceId': 'de2ac956-c885-4221-af25-a21ec6ebcf27', 'faceRectangle': {'top': 45, 'left': 96, 'width': 90, 'height': 90}}]


21. En este caso utilizaremos el servicio de **identify** para comparar este nuevo rostro respecto a los usuarios de la base de datos generada previamente.

### Face - Identify

1-to-many identification to find the closest matches of the specific query person face from a person group or large person group.

For each face in the faceIds array, Face Identify will compute similarities between the query face and all the faces in the person group (given by personGroupId) or large person group (given by largePersonGroupId), and return candidate person(s) for that face ranked by similarity confidence. The person group/large person group should be trained to make it ready for identification. See more in PersonGroup - Train and LargePersonGroup - Train.

Remarks:
- The algorithm allows more than one face to be identified independently at the same request, but no more than 10 faces.
- Each person in the person group/large person group could have more than one face, but no more than 248 faces.
- Higher face image quality means better identification precision. Please consider high-quality faces: frontal, clear, and face size is 200x200 pixels (100 pixels between eyes) or bigger.
- Number of candidates returned is restricted by maxNumOfCandidatesReturned and confidenceThreshold. If no person is identified, the returned candidates will be an empty array.
- Try Face - Find Similar when you need to find similar faces from a face list/large face list instead of a person group/large person group.
- The 'recognitionModel' associated with the query faces' faceIds should be the same as the 'recognitionModel' used by the target person group or large person group.

#### Http Method

POST

#### Request URL

https://{endpoint}/face/v1.0/identify

#### Request headers

| Field | Type | Description |
|:--- |:---:|:--- |
| Content-Type | string | Media type of the body sent to the API. |
| Ocp-Apim-Subscription-Key | string | Subscription key which provides access to this API. Found in your Cognitive Services accounts. |

#### Request body

JSON fields in request body:

| Field | Type | Description |
|:--- |:---:|:--- |
| faceIds | Array | Array of query faces faceIds, created by the Face - Detect. Each of the faces are identified independently. The valid number of faceIds is between [1, 10]. |
| personGroupId | String | personGroupId of the target person group, created by PersonGroup - Create. Parameter personGroupId and largePersonGroupId should not be provided at the same time. |
| largePersonGroupId | String | largePersonGroupId of the target large person group, created by LargePersonGroup - Create. Parameter personGroupId and largePersonGroupId should not be provided at the same time. |
| maxNumOfCandidatesReturned (optional) | Number | The range of maxNumOfCandidatesReturned is between 1 and 100 (default is 10). |
| confidenceThreshold (optional) | Number | Optional parameter. Customized identification confidence threshold, in the range of [0, 1]. Advanced user can tweak this value to override default internal threshold for better precision on their scenario data. Note there is no guarantee of this threshold value working on other data and after algorithm updates. |

In [113]:
response = requests.request(
    #method
    'POST',
    #url
    BASE_URL + '/identify',
    #json
    json = {'faceIds': [faceId],
            'largePersonGroupId': nombre_base},
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

response.status_code

200

22. Obtenemos una respuesta **200**, lo cual implica que el proceso de comparación finalizó de forma correcta. Ahora solo falta revisa si encontró o no alguna similitud.

In [115]:
print(response.json())

id_similar = response.json()[0]['candidates'][0]['personId']

[{'faceId': 'de2ac956-c885-4221-af25-a21ec6ebcf27', 'candidates': [{'personId': '48819917-a770-47d8-9c52-a807f9b7aee5', 'confidence': 0.90931}]}]


23. La respuesta del API nos indica que existe una similitud del 0.91 respecto a un candidato. Sin embargo nos entrega el Id del usuario. Por lo que tendrémos que extraer la información del nombre. Usaremos el **person list** para este proposito.

In [123]:
response = requests.request(
    #method
    'GET',
    #url
    BASE_URL + '/largepersongroups/' + nombre_base + '/persons',
    #headers
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': KEY}
)

for usuario in response.json():
    if usuario['personId'] in id_similar:
        print(usuario['name'])

emilia_clarke


---

# Ejercicio

1. Usar las imágenes en la carpeta de prueba y verficar si encuentra similitudes con los usuarios dados de alta.
    - ¿Logró identificar todos los rostros de las imágenes utilizadas?
    - ¿Encontró similitudes en todos ellos?

2. Buscar más imagenes en internet y actualizar la base de datos con nuevos usuarios y/o más imágenes de los usuarios existentes.

---

![](imgs/end.png)